In [51]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import ipywidgets as widgets
from matplotlib.cm import viridis
from matplotlib.cm import Spectral
from matplotlib.cm import autumn
from matplotlib.colors import to_hex

import gmaps
import gmaps.geojson_geometries
import gmaps.datasets
import os
import json
import requests
import pprint as pp

import csv

# Import API key
from config import g_key

In [76]:
NYC_df = pd.read_csv('resources/FixedNYClisting.csv')
NYC_df['price'] = NYC_df['price'].map(lambda x: str(x)[1:])
NYC_df['price'] = NYC_df['price'].map(lambda x: x.replace(',', ''))
NYC_df['review_scores_rating'] = NYC_df['review_scores_rating']/10
#NYC_df['Value'] = NYC_df['price']/ (NYC_df['review_scores_rating']+NYC_df['review_scores_location'])
NYC_df['price'] = NYC_df['price'].astype('float')
NYC_df.head()

,Unnamed: 0,id,name,neighbourhood,latitude,longitude,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month
0,0,2595,Skylit Midtown Castle,Midtown,40.75362,-73.98377,225.0,46,9.5,10.0,0.39
1,2,3831,Cozy Entire Floor of Brownstone,Brooklyn,40.68514,-73.95976,89.0,274,9.0,10.0,4.64
2,3,5022,Entire Apt: Spacious Studio/Loft by central park,East Harlem,40.79851,-73.94399,80.0,9,9.3,9.0,0.10
3,4,5099,Large Cozy 1 BR Apartment In Midtown East,Midtown East,40.74767,-73.97500,200.0,75,8.9,9.0,0.60
4,5,5121,BlissArtsSpace!,Brooklyn,40.68688,-73.95596,60.0,49,9.0,9.0,0.39


In [71]:
# Grab ratings
neighborhood_ratings = NYC_df[['neighbourhood','review_scores_location']]
neighborhood_prices = NYC_df[['neighbourhood','price']]
neighborhood_bookings = NYC_df[['neighbourhood','reviews_per_month']]
neighborhood_bookings

,neighbourhood,reviews_per_month
0,Midtown,0.39
1,Brooklyn,4.64
2,East Harlem,0.10
3,Midtown East,0.60
4,Brooklyn,0.39
5,Manhattan,3.48
6,Upper West Side,0.98
7,Manhattan,0.74
8,Lower East Side,1.33
9,Manhattan,0.42


In [69]:
neighborhood_ratings.dropna()
neighborhood_prices.dropna()
NYC_loc_scores_df = neighborhood_ratings.groupby('neighbourhood').mean()
NYC_loc_scores_df.head()
NYC_price_df = neighborhood_prices.groupby('neighbourhood').mean()
NYC_price_df.head()

,price
neighbourhood,
Allerton,67.454545
Alphabet City,156.677130
Annadale,121.000000
Arden Heights,70.000000
Arrochar,99.000000


In [8]:
NYC_loc_scores_df.to_csv(r'resources/nyc_loc_scores.csv')

In [9]:
csv_data = pd.read_csv('resources/nyc_loc_scores.csv')
csv_data.head()

,neighbourhood,review_scores_location
0,Allerton,9.636364
1,Alphabet City,9.695067
2,Annadale,10.000000
3,Arden Heights,10.000000
4,Arrochar,8.000000


In [43]:
# Authenticate gmaps
gmaps.configure(api_key=g_key)

nyc_coordinates = [(40.7128, 74.0060)]

# Load in NYC GeoJSON
with open('resources/custom-pedia-cities-nyc-Mar2018.json') as f:
    neighborhoods_geojson = json.load(f)

# Turn Neighborhoods Rating DataFrame into Dictionary
avg_location_dict = NYC_loc_scores_df.to_dict()
neighborhood2gini = avg_location_dict['review_scores_location']
min_gini = min(neighborhood2gini.values())
max_gini = max(neighborhood2gini.values())
gini_range = max_gini - min_gini

def calculate_color(gini):
    """
    Convert the GINI coefficient to a color
    """
    # make gini a number between 0 and 1
    normalized_gini = (gini - min_gini) / gini_range

    # invert gini so that high inequality gives dark color
    inverse_gini = 1.0 - normalized_gini

    # transform the gini coefficient to a matplotlib color
    mpl_color = viridis(inverse_gini)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

# Calculate a color for each GeoJSON feature
colors = []
unique_colors = {}
legend_gini = [10.0, 9.5, 9.0, 8.5, 8.0, 7.5, 0]
legend_entries = []
#add black for null values
legend_entries.append('<span style="color: #000000; font-size: 2em; vertical-align: bottom;">&#9679;</span> 0.0')
# Some CSS hackery to draw a circle of that color next to the year
legend_template = '<span style="color: {color}; font-size: 2em; vertical-align: bottom;">&#9679;</span> {value}'

for feature in neighborhoods_geojson['features']:
    neighborhood_name = feature['properties']['neighborhood']
    try:
        gini = neighborhood2gini[neighborhood_name]
        color = calculate_color(gini)
        if gini in legend_gini:
            if color not in unique_colors.keys():
                unique_colors[color] = gini
                legend_entries.append(legend_template.format(color=color, value=gini))
    except KeyError:
        # no GINI for that country: return default color
        color = (0, 0, 0, 0.9)
    colors.append(color)


# fix null value            
unique_colors[(0, 0, 0, 0.9)] = 0


def draw_map_with_legend():
    figure = gmaps.figure(center=(40.7, -74.0), zoom_level = 9.6)
    
    gini_layer = gmaps.geojson_layer(
    neighborhoods_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.9)
    figure.add_layer(gini_layer)
    
    legend = widgets.HBox(
        [widgets.HTML(legend_entry) for legend_entry in legend_entries],
        layout=widgets.Layout(width='100%', justify_content='space-between')
    )
    return widgets.VBox([figure, legend])

draw_map_with_legend()

{'price': {'Allerton': 67.45454545454545,
  'Alphabet City': 156.67713004484304,
  'Annadale': 121.0,
  'Arden Heights': 70.0,
  'Arrochar': 99.0,
  'Astoria': 117.38077634011091,
  'Bath Beach': 105.0,
  'Battery Park City': 147.7,
  'Bay Ridge': 111.63917525773196,
  'Baychester': 81.03703703703704,
  'Bayside': 84.8,
  'Bedford Park': 84.36363636363636,
  'Bedford-Stuyvesant': 108.3103448275862,
  'Belmont': 52.0,
  'Bensonhurst': 73.4,
  'Bergen Beach': 85.0,
  'Boerum Hill': 172.289156626506,
  'Borough Park': 63.6875,
  'Brighton Beach': 93.95652173913044,
  'Bronxdale': 54.714285714285715,
  'Brooklyn': 122.61401439188393,
  'Brooklyn Heights': 231.41176470588235,
  'Brooklyn Navy Yard': 125.34615384615384,
  'Brownsville': 66.0,
  'Bushwick': 84.91592356687899,
  'Canarsie': 100.2,
  'Carroll Gardens': 168.9857142857143,
  'Castle Hill ': 86.75,
  'Castleton Corners': 299.0,
  'Chelsea': 207.61964735516372,
  'Chinatown': 171.76377952755905,
  'City Island': 97.23076923076923,


In [58]:
# Authenticate gmaps
gmaps.configure(api_key=g_key)

nyc_coordinates = [(40.7128, 74.0060)]

# Load in NYC GeoJSON
with open('resources/custom-pedia-cities-nyc-Mar2018.json') as f:
    neighborhoods_geojson = json.load(f)

# Turn Neighborhoods Rating DataFrame into Dictionary
price_dict = NYC_price_df.to_dict()
neighborhood2gini = price_dict['price']
min_gini = min(neighborhood2gini.values())
max_gini = max(neighborhood2gini.values())
gini_range = max_gini - min_gini

def calculate_color(gini):
    """
    Convert the GINI coefficient to a color
    """
    # make gini a number between 0 and 1
    normalized_gini = (gini - min_gini) / gini_range

    # invert gini so that high inequality gives dark color
    inverse_gini = 1.0 - normalized_gini

    # transform the gini coefficient to a matplotlib color
    mpl_color = autumn(inverse_gini)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

# Calculate a color for each GeoJSON feature
colors = []
unique_colors = {}

legend_gini = [10.0, 9.5, 9.0, 8.5, 8.0, 7.5, 0]
legend_entries = []
#add black for null values
legend_entries.append('<span style="color: #ffffff; font-size: 2em; vertical-align: bottom;">&#9679;</span> 0.0')
# Some CSS hackery to draw a circle of that color next to the year
legend_template = '<span style="color: {color}; font-size: 2em; vertical-align: bottom;">&#9679;</span> {value}'

for feature in neighborhoods_geojson['features']:
    neighborhood_name = feature['properties']['neighborhood']
    try:
        gini = neighborhood2gini[neighborhood_name]
        color = calculate_color(gini)
        if gini in legend_gini:
            if color not in unique_colors.keys():
                unique_colors[color] = gini
                legend_entries.append(legend_template.format(color=color, value=gini))
    except KeyError:
        # no GINI for that country: return default color
        color = (255, 255, 255, 0.5)
    colors.append(color)


# fix null value            
unique_colors[(255, 255, 255, 0.9)] = 0


def draw_map_with_legend():
    figure = gmaps.figure(center=(40.7, -74.0), zoom_level = 9.6)
    
    gini_layer = gmaps.geojson_layer(
    neighborhoods_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.9)
    figure.add_layer(gini_layer)
    
    legend = widgets.HBox(
        [widgets.HTML(legend_entry) for legend_entry in legend_entries],
        layout=widgets.Layout(width='100%', justify_content='space-between')
    )
    return widgets.VBox([figure, legend])

draw_map_with_legend()

In [67]:
neighborhood_price_dict = price_dict['price']

neighborhood_price_dict

{'Allerton': 67.45454545454545,
 'Alphabet City': 156.67713004484304,
 'Annadale': 121.0,
 'Arden Heights': 70.0,
 'Arrochar': 99.0,
 'Astoria': 117.38077634011091,
 'Bath Beach': 105.0,
 'Battery Park City': 147.7,
 'Bay Ridge': 111.63917525773196,
 'Baychester': 81.03703703703704,
 'Bayside': 84.8,
 'Bedford Park': 84.36363636363636,
 'Bedford-Stuyvesant': 108.3103448275862,
 'Belmont': 52.0,
 'Bensonhurst': 73.4,
 'Bergen Beach': 85.0,
 'Boerum Hill': 172.289156626506,
 'Borough Park': 63.6875,
 'Brighton Beach': 93.95652173913044,
 'Bronxdale': 54.714285714285715,
 'Brooklyn': 122.61401439188393,
 'Brooklyn Heights': 231.41176470588235,
 'Brooklyn Navy Yard': 125.34615384615384,
 'Brownsville': 66.0,
 'Bushwick': 84.91592356687899,
 'Canarsie': 100.2,
 'Carroll Gardens': 168.9857142857143,
 'Castle Hill ': 86.75,
 'Castleton Corners': 299.0,
 'Chelsea': 207.61964735516372,
 'Chinatown': 171.76377952755905,
 'City Island': 97.23076923076923,
 'Civic Center': 130.5,
 'Claremont': 48.

In [68]:
sorted(neighborhood_price_dict.items(), key=lambda x: x[1], reverse=True)

[('Flatiron District', 336.6951219512195),
 ('Tribeca', 328.86842105263156),
 ('Noho', 316.2142857142857),
 ('Castleton Corners', 299.0),
 ('Soho', 253.3734939759036),
 ('Financial District', 246.2756183745583),
 ('Cobble Hill', 243.1904761904762),
 ('West Village', 236.61654135338347),
 ('Midtown East', 234.13229571984436),
 ('Whitestone', 233.4),
 ('Brooklyn Heights', 231.41176470588235),
 ('Midtown', 215.00531914893617),
 ('Sea Gate', 215.0),
 ('Murray Hill', 213.21428571428572),
 ('Gramercy Park', 210.57894736842104),
 ('Chelsea', 207.61964735516372),
 ('Times Square/Theatre District', 201.8421052631579),
 ('South Beach', 193.125),
 ('Nolita', 191.95833333333334),
 ('Union Square', 191.5),
 ('Little Italy', 190.5),
 ('Kips Bay', 188.59649122807016),
 ("Hell's Kitchen", 188.4728132387707),
 ('Greenwich Village', 186.35238095238094),
 ('Huguenot', 185.0),
 ('Upper West Side', 183.78503184713375),
 ('East Village', 182.26739926739927),
 ('Manhattan', 181.33947368421053),
 ('Hudson Squ

In [87]:
NYC_df['Value for Renter'] = (NYC_df['review_scores_rating']+NYC_df['review_scores_location'])/NYC_df['price']
NYC_df.head()




,Unnamed: 0,id,name,neighbourhood,latitude,longitude,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month,Value for Seller,Value for Renter
0,0,2595,Skylit Midtown Castle,Midtown,40.75362,-73.98377,225.0,46,9.5,10.0,0.39,11.538462,0.086667
1,2,3831,Cozy Entire Floor of Brownstone,Brooklyn,40.68514,-73.95976,89.0,274,9.0,10.0,4.64,4.684211,0.213483
2,3,5022,Entire Apt: Spacious Studio/Loft by central park,East Harlem,40.79851,-73.94399,80.0,9,9.3,9.0,0.10,4.371585,0.228750
3,4,5099,Large Cozy 1 BR Apartment In Midtown East,Midtown East,40.74767,-73.97500,200.0,75,8.9,9.0,0.60,11.173184,0.089500
4,5,5121,BlissArtsSpace!,Brooklyn,40.68688,-73.95596,60.0,49,9.0,9.0,0.39,3.333333,0.300000


In [88]:
NYC_df.sort_values(by='Value for Renter', ascending=False)

,Unnamed: 0,id,name,neighbourhood,latitude,longitude,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month,Value for Seller,Value for Renter
20653,25333,20639792,Contemporary bedroom in brownstone with nice view,Bedford-Stuyvesant,40.68279,-73.91170,0.0,101,9.2,9.0,4.46,0.000000,inf
20638,25316,20624541,Modern apartment in the heart of Williamsburg,Williamsburg,40.70838,-73.94645,0.0,3,8.7,9.0,0.14,0.000000,inf
20516,25172,20523843,"MARTIAL LOFT 3: REDEMPTION (upstairs, 2nd room)",Bushwick,40.69467,-73.92433,0.0,16,9.4,10.0,0.68,0.000000,inf
20652,25332,20639628,Spacious comfortable master bedroom with nice ...,Brooklyn,40.68173,-73.91342,0.0,98,9.3,9.0,4.32,0.000000,inf
18522,22745,18750597,"Huge Brooklyn Brownstone Living, Close to it all.",Brooklyn,40.69023,-73.95428,0.0,1,8.0,10.0,0.05,0.000000,inf
21514,26359,21291569,Coliving in Brooklyn! Modern design / Shared room,Brooklyn,40.69211,-73.90670,0.0,2,7.0,8.0,0.10,0.000000,inf
20654,25334,20639914,Cozy yet spacious private brownstone bedroom,Bedford-Stuyvesant,40.68258,-73.91284,0.0,102,9.1,9.0,4.48,0.000000,inf
20618,25290,20608117,"Sunny, Quiet Room in Greenpoint",Brooklyn,40.72462,-73.94072,0.0,12,9.8,10.0,0.51,0.000000,inf
21532,26382,21304320,Best Coliving space ever! Shared room.,Bushwick,40.69166,-73.90928,0.0,5,9.6,10.0,0.25,0.000000,inf
28515,34875,28270998,"Charming, bright and brand new Bed-Stuy home",Bedford-Stuyvesant,40.69508,-73.95164,10.0,7,10.0,10.0,0.65,0.500000,2.000000
